# DwC Occurrences. Keister Zooplankton Hood Canal 2012-13 data

University of Washington Pelagic Hypoxia Hood Canal project, Zooplankton dataset.

5/4, 4/5-4, 3/28,7, 2023. 2022-3-14, 1-12

**BRIEFLY DESCRIBE WHAT THIS NOTEBOOK DOES**

In [1]:
from datetime import datetime
from pathlib import Path
import uuid

import numpy as np
import pandas as pd

from data_preprocess import read_and_parse_sourcedata

In [2]:
data_pth = Path(".")

## Settings

Set to `True` when debugging. `csv` ﬁles will not be exported when `debug_no_csvexport = True`

In [3]:
debug_no_csvexport = False

## Process JSON file containing common mappings and strings

In [4]:
with open(data_pth / 'common_mappings.json') as f:
    common_mappings = json.load(f)

In [5]:
DatasetCode = common_mappings['datasetcode']
sex_dwciri_terms = common_mappings['sex_dwciri_terms']
life_stage_mappings = common_mappings['life_stage_mappings']

# TODO: Create two mappings from life_stage_mappings, for lifeStage and dwciri:lifeStage
lifeStage_mapping = {k:v[0] for k,v in life_stage_mappings.items()}
lifeStage_dwciri_terms = {k:v[1] for k,v in life_stage_mappings.items()}

## Read the data

### Read the pre-processed csv file

`usecols` defines the columns that will be kept and the order in which they'll be organized

In [6]:
usecols = ['sample_code', 'species', 'life_history_stage', 'lhs_0', 'lhs_1']
occursource_df = read_and_parse_sourcedata()[usecols]

In [7]:
len(occursource_df)

6884

In [8]:
occursource_df.head()

,sample_code,species,life_history_stage,lhs_0,lhs_1
0,20131003DBDm2_200,ACARTIA,3;_CIII,3,CIII
1,20130906DBiDm1_200,ACARTIA,5;_CV,5,CV
2,20131003DBDm1_200,ACARTIA,Female;_Adult,Female,Adult
3,20131003DBDm1_200,ACARTIA,Male;_Adult,Male,Adult
4,20120614DBDm3_200,ACARTIA_CLAUSI,Female;_Adult,Female,Adult


## Merge resolved taxonomy from taxonomy csv

In [9]:
taxonomy_df = pd.read_csv(
    data_pth / "intermediate_DwC_taxonomy.csv"
)

In [10]:
occursource_df = occursource_df.merge(
    taxonomy_df, 
    how='inner', 
    left_on='species',
    right_on='verbatimIdentification'
)

In [11]:
len(occursource_df)

6867

In [12]:
occursource_df.head()

,sample_code,species,life_history_stage,lhs_0,lhs_1,scientificName,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,20131003DBDm2_200,ACARTIA,3;_CIII,3,CIII,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
1,20130906DBiDm1_200,ACARTIA,5;_CV,5,CV,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
2,20131003DBDm1_200,ACARTIA,Female;_Adult,Female,Adult,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
3,20131003DBDm1_200,ACARTIA,Male;_Adult,Male,Adult,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
4,20130905DUNm1_200,ACARTIA,5;_CV,5,CV,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA


## Create and populate `occurrence_df` dataframe

### Add `occurrenceID`, `basisOfRecord`, `occurrenceStatus`

In [13]:
occurrence_df = occursource_df.copy()

In [14]:
occurrence_df.insert(1, 'occurrenceID', [uuid.uuid4() for i in range(len(occurrence_df))])
occurrence_df.insert(2, 'basisOfRecord', 'MaterialSample')
occurrence_df.insert(3, 'occurrenceStatus', 'present')

occurrence_df.rename(columns={'sample_code':'eventID'}, inplace=True)

In [15]:
len(occurrence_df)

6867

In [16]:
occurrence_df.head(10)

,eventID,occurrenceID,basisOfRecord,occurrenceStatus,species,life_history_stage,lhs_0,lhs_1,scientificName,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,20131003DBDm2_200,13410b32-6144-4461-8a91-43f2b341375f,MaterialSample,present,ACARTIA,3;_CIII,3,CIII,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
1,20130906DBiDm1_200,851399bb-af9e-4293-887c-73925d0018b3,MaterialSample,present,ACARTIA,5;_CV,5,CV,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
2,20131003DBDm1_200,649762f7-673b-459a-8403-b1c66fa8a64a,MaterialSample,present,ACARTIA,Female;_Adult,Female,Adult,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
3,20131003DBDm1_200,14ff9677-c729-447a-86f0-de22e7052342,MaterialSample,present,ACARTIA,Male;_Adult,Male,Adult,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
4,20130905DUNm1_200,8159e432-fa59-42b5-a484-40ce6bd84e4d,MaterialSample,present,ACARTIA,5;_CV,5,CV,Acartia,urn:lsid:marinespecies.org:taxname:104108,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Dana, 1846",ACARTIA
5,20120614DBDm3_200,dcfc768c-61fc-42cb-9f7c-cbdca7777dc4,MaterialSample,present,ACARTIA_CLAUSI,Female;_Adult,Female,Adult,Acartia clausi,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
6,20120614DBDm3_200,9466b775-f0e5-4e1d-9d63-6b55b9a8431e,MaterialSample,present,ACARTIA_CLAUSI,Male;_Adult,Male,Adult,Acartia clausi,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
7,20120614DBDm4_200,5dd5bdea-0739-41a3-8314-97472742c35d,MaterialSample,present,ACARTIA_CLAUSI,Female;_Adult,Female,Adult,Acartia clausi,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
8,20120712DBDm1_200,aaa002bd-72d2-44c2-9621-35193bc30669,MaterialSample,present,ACARTIA_CLAUSI,5;_CV,5,CV,Acartia clausi,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
9,20121004DBDm4_200,64c1070d-2b52-481a-ab09-6bb9ff0c3b3b,MaterialSample,present,ACARTIA_CLAUSI,Female;_Adult,Female,Adult,Acartia clausi,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI


### Map `life_history_stage` into `sex` and `lifeStage`

In [17]:
# dwc standard columns
occurrence_df.insert(
    4, 'sex', 
    occurrence_df['lhs_0'].apply(
        lambda s: s.lower() if s in ['Female', 'Male'] else 'indeterminate'
    )
)
occurrence_df.insert(
    5, 'lifeStage', 
    occurrence_df['lhs_0'].apply(lambda s: lifeStage_mapping[s])
)

# dwciri columns
nercvocabs_url = "http://vocab.nerc.ac.uk/collection/"
occurrence_df.insert(
    6, 'dwciri:sex', 
    occurrence_df['sex'].apply(lambda s: nercvocabs_url + sex_dwciri_terms[s])
)
occurrence_df.insert(
    7, 'dwciri:lifeStage', 
    occurrence_df['lhs_0'].apply(
        lambda s: None if lifeStage_dwciri_terms[s] is None 
        else nercvocabs_url + lifeStage_dwciri_terms[s]
    )
)

In [18]:
occurrence_df = (
    occurrence_df
    .sort_values(by=['eventID', 'scientificName', 'lifeStage', 'sex'])
    .reset_index(drop=True)
)

In [19]:
occurrence_df.head(5)

,eventID,occurrenceID,basisOfRecord,occurrenceStatus,sex,lifeStage,dwciri:sex,dwciri:lifeStage,species,life_history_stage,...,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,20120611UNDm1_200,0766710d-10e0-4b3f-966b-499bc816caff,MaterialSample,present,female,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,ACARTIA_CLAUSI,Female;_Adult,...,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
1,20120611UNDm1_200,9085cf72-19bc-4978-97ff-883b78668278,MaterialSample,present,male,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,ACARTIA_CLAUSI,Male;_Adult,...,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
2,20120611UNDm1_200,4e8c8e51-859e-478d-b548-aac8bf5c3e19,MaterialSample,present,indeterminate,copepodites C5,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,ACARTIA_CLAUSI,5;_CV,...,urn:lsid:marinespecies.org:taxname:104251,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Giesbrecht, 1889",ACARTIA_CLAUSI
3,20120611UNDm1_200,3541fbe0-01bb-41e7-9f4f-969934b0c980,MaterialSample,present,indeterminate,medusae,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,AGLANTHA,Medusa,...,urn:lsid:marinespecies.org:taxname:117212,Genus,Animalia,Cnidaria,Hydrozoa,Trachymedusae,Rhopalonematidae,Aglantha,"Haeckel, 1879",AGLANTHA
4,20120611UNDm1_200,fc2afcf0-fe9b-4225-a6dd-8060abae62da,MaterialSample,present,indeterminate,veliger,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,BIVALVIA,Veliger,...,urn:lsid:marinespecies.org:taxname:105,Class,Animalia,Mollusca,Bivalvia,NaN,NaN,NaN,"Linnaeus, 1758",BIVALVIA


## Export intermediate table for `life_history_stage` matching to csv

In [20]:
if not debug_no_csvexport:
    occurrence_df[['occurrenceID', 'life_history_stage']].to_csv(
        data_pth / 'intermediate_DwC_occurrence_life_history_stage.csv', index=False
    )

## Export `occurrence_df` to csv

### Cleanup

In [21]:
occurrence_df.drop(columns=['species', 'life_history_stage', 'lhs_0', 'lhs_1'], inplace=True)

if not debug_no_csvexport:
    occurrence_df.to_csv(data_pth / 'aligned_csvs' / 'DwC_occurrence.csv', index=False)

## Package versions

In [22]:
print(
    f"{datetime.utcnow()} +00:00\n"
    f"pandas: {pd.__version__}"
)

2023-05-04 08:10:59.199071 +00:00
pandas: 1.5.3
